## OPRO-v2

Goals:
1. Write the input -> output transformation (what is it?)
2. Template the whole thing
3. Inherit from optoprime_v2

In [2]:
from dotenv import load_dotenv
load_dotenv() 

True

In [5]:
from opto import trace
from opto.trace import node, bundle
from opto.optimizers.opro_v2 import OPROv2
from opto.trace.operators import call_llm

In [6]:
num_1 = node(1, trainable=True)
optimizer = OPROv2([num_1],use_json_object_format=False,
                ignore_extraction_error=False,
                include_example=True)

@bundle()
def propose_solution(x):
    """
    Propose a solution to the given prompt using the input.
    """
    return x + 1

In [7]:
result = propose_solution(num_1)

optimizer.zero_feedback()
optimizer.backward(result, 'make this number bigger')

In [8]:
summary = optimizer.summarize()
part1, part2 = optimizer.construct_prompt(summary)

part1 = optimizer.replace_symbols(part1, optimizer.prompt_symbols)
part2 = optimizer.replace_symbols(part2, optimizer.prompt_symbols)

In [9]:
print(part1)


You're tasked to change the proposed solution according to feedback.

Specifically, a problem will be composed of the following parts:
- #Instruction: the instruction which describes the things you need to do or the question you should answer.
- #Variables: the proposed solution that you can change/tweak (trainable).
- #Feedback: the feedback about the solution.

If `data_type` is `code`, it means `value` is the source code of a python code, which may include docstring and definitions.

Output_format: Your output should be in the following XML/HTML format:

```

<reasoning>
reasoning
</reasoning>
<variable>
<name>variable_name</name>
<value>
value
</value>
</variable>

```

In <reasoning>, explain the problem: 1. what the #Instruction means 2. what the #Feedback means to #Variables considering how #Variables follow #Instruction. 3. Reasoning about the suggested changes in #Variables (if needed) and the expected result.

If you need to suggest a change in the values of #Variables, writ

In [10]:
print(part2)


Now you see problem instance:


# Instruction
Propose a new solution that will incorporate the feedback.

# Solution
<solution name="int0" type="int">
<value>
1
</value>
</solution>


# Feedback
make this number bigger



What are your revised solutions on int0?

Your response:



In [11]:
messages = [
    {"role": "system", "content": part1},
    {"role": "user", "content": part2},
]

response = optimizer.llm(messages=messages)
response = response.choices[0].message.content
reasoning = response

In [12]:
suggestion = optimizer.extract_llm_suggestion(response)

In [13]:
suggestion

{'reasoning': 'The #Instruction requests a new solution that incorporates the given feedback into the proposed solution. The #Variables section includes an integer variable "int0" with the current value set to 1. The feedback states that this number should be made "bigger." Thus, the current value does not meet the feedback requirement, and I should change it to a larger integer value to comply with the feedback. A simple increment will suffice, so I will propose changing "int0" from 1 to 2.',
 'variables': {'int0': '2'}}

In [14]:
response

'```\n\n<reasoning>\nThe #Instruction requests a new solution that incorporates the given feedback into the proposed solution. The #Variables section includes an integer variable "int0" with the current value set to 1. The feedback states that this number should be made "bigger." Thus, the current value does not meet the feedback requirement, and I should change it to a larger integer value to comply with the feedback. A simple increment will suffice, so I will propose changing "int0" from 1 to 2.\n</reasoning>\n<variable>\n<name>int0</name>\n<value>\n2\n</value>\n</variable>\n\n```'

In [7]:
import opto.trace as trace 

@trace.bundle()
async def a(x, y):
    return x + y

In [8]:
a.__signature__

<Signature (x, y)>

In [9]:
a.__code__

<code object a at 0x107d698f0, file "/var/folders/mk/z770g0p92jvf3xl17zxnt0v40000gn/T/ipykernel_55615/696956115.py", line 3>

In [12]:
import inspect 
inspect.iscoroutinefunction(a)

True

In [3]:
import inspect
inspect.signature(a)

<Signature (x, y)>

In [6]:
hasattr(a, "__text_signature__")

False

In [5]:
callable(a)

True

In [7]:
import inspect
from inspect import Signature, Parameter

class MyCallable:
    def __init__(self, func):
        self.func = func

    def __call__(self, a, b=42):
        return self.func(a, b)

# Define signature explicitly
sig = Signature([
    Parameter('a', kind=Parameter.POSITIONAL_OR_KEYWORD),
    Parameter('b', kind=Parameter.POSITIONAL_OR_KEYWORD, default=42),
])
MyCallable.__signature__ = sig

def add(a, b):
    return a + b

wrapped = MyCallable(add)
print(inspect.signature(wrapped))  # → (a, b=42)

(a, b=42)


In [8]:
add.__code__

<code object add at 0x10439e4a0, file "/var/folders/mk/z770g0p92jvf3xl17zxnt0v40000gn/T/ipykernel_49726/3307697563.py", line 18>